Hämta data från Wikidata med Python
* [video](https://youtu.be/ZcvV4det8tA)
* denna [Notebook](https://github.com/salgo60/SDGMapperSweden/blob/main/Notebook/GetWikidata.ipynb)  
 
 
* GITHUB [Issue](https://github.com/Ackuq/partiguiden/issues/1249) där det diskuteras Länkade data, Wikidata med koppling partiernas program

In [9]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """#title: partier i Sverige
SELECT distinct ?party ?partyLabel ?wikipedia ?www WHERE {

  VALUES ?member {
    wd:Q10655178 
  }
  ?person wdt:P39 ?member.
  ?person wdt:P102 ?party.
  ?party wdt:P17 wd:Q34.
  ?party wdt:P856 ?www
  minus {?party wdt:P576 ?xxx}
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv". }
  
   OPTIONAL {
      ?wikipedia schema:about ?party .
      ?wikipedia schema:inLanguage "sv" .
      ?wikipedia schema:isPartOf <https://sv.wikipedia.org/> .
    }
} GROUP BY  ?party ?partyLabel ?wikipedia ?www
order by ?partyLabel"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result)


{'party': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q50383811'}, 'partyLabel': {'xml:lang': 'sv', 'type': 'literal', 'value': 'Alternativ för Sverige'}, 'wikipedia': {'type': 'uri', 'value': 'https://sv.wikipedia.org/wiki/Alternativ_f%C3%B6r_Sverige'}, 'www': {'type': 'uri', 'value': 'https://alternativforsverige.se/'}}
{'party': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q110832'}, 'partyLabel': {'xml:lang': 'sv', 'type': 'literal', 'value': 'Centerpartiet'}, 'wikipedia': {'type': 'uri', 'value': 'https://sv.wikipedia.org/wiki/Centerpartiet'}, 'www': {'type': 'uri', 'value': 'http://www.centerpartiet.se/'}}
{'party': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q111449676'}, 'partyLabel': {'xml:lang': 'sv', 'type': 'literal', 'value': 'Klimatalliansen'}, 'www': {'type': 'uri', 'value': 'https://klimatalliansen.nu/'}}
{'party': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q213654'}, 'partyLabel': {'xml:lang': 'sv', 'type': 'literal',

## Samma fråga men till pandas

In [10]:
import json
import pandas as pd
def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

dfPartier = get_sparql_dataframe(endpoint_url, query)

dfPartier

,party,partyLabel,wikipedia,www
0,http://www.wikidata.org/entity/Q50383811,Alternativ för Sverige,https://sv.wikipedia.org/wiki/Alternativ_f%C3%...,https://alternativforsverige.se/
1,http://www.wikidata.org/entity/Q110832,Centerpartiet,https://sv.wikipedia.org/wiki/Centerpartiet,http://www.centerpartiet.se/
2,http://www.wikidata.org/entity/Q111449676,Klimatalliansen,None,https://klimatalliansen.nu/
3,http://www.wikidata.org/entity/Q213654,Kristdemokraterna,https://sv.wikipedia.org/wiki/Kristdemokratern...,http://www.kristdemokraterna.se/
4,http://www.wikidata.org/entity/Q110857,Liberalerna,https://sv.wikipedia.org/wiki/Liberalerna,https://www.liberalerna.se/
5,http://www.wikidata.org/entity/Q213451,Miljöpartiet,https://sv.wikipedia.org/wiki/Milj%C3%B6partiet,http://www.mp.se/
6,http://www.wikidata.org/entity/Q110843,Moderaterna,https://sv.wikipedia.org/wiki/Moderaterna,https://moderaterna.se/
7,http://www.wikidata.org/entity/Q61791721,Partiet Vändpunkt,https://sv.wikipedia.org/wiki/Partiet_V%C3%A4n...,https://partietvandpunkt.se
8,http://www.wikidata.org/entity/Q7140617,Partipolitiskt obundna i Svenska kyrkan,https://sv.wikipedia.org/wiki/Partipolitiskt_o...,https://www.posk.se/
9,http://www.wikidata.org/entity/Q105112,Socialdemokraterna,https://sv.wikipedia.org/wiki/Socialdemokrater...,http://www.socialdemokraterna.se/
